In [392]:
from bs4 import BeautifulSoup
import pickle
import time
import requests
import csv
import re
from dateutil import parser
from datetime import timedelta
from datetime import datetime
import random
import os
import pandas as pd
import numpy as np

In [ ]:
# Scrape a List of movies released in 2015 with wikipedia articles (and the links)

response = requests.get("https://en.wikipedia.org/wiki/2015_in_film")
soup = BeautifulSoup(response.text, "lxml")
tables = soup.find_all("table", class_="wikitable")

In [ ]:
movies = {}
for table in tables[3:7]: # The tables I want are 4 - 7
    for movie in table.find_all("i"):
        movie = movie.find("a")
        if movie is not None:
            url = movie.get("href").split("/")[-1].encode("utf-8")
            text = movie.text.encode("utf-8")
            movies[text] = {"url":url}

In [ ]:
movies

In [ ]:
def release_dates(soup):
    try:
        node = soup.find(text=re.compile("Release dates"))
        node = node.parent.parent.findNextSibling()
        return [date.text for date in node.find_all("li") if len(date.text) > 0]
    except:
        return None

def budget(soup):
    try:
        node = soup.find(text=re.compile("Budget"))
        node = node.parent.findNextSibling()
        return node.text.split("[")[0]
    except:
        return None
    
def box_office(soup):
    try:
        node = soup.find(text=re.compile("Box office"))
        node = node.parent.findNextSibling()
        return node.text.split("[")[0]
    except:
        return None
    
        

In [ ]:
for movie, data in movies.iteritems():
    print movie
    url = data['url']
    ## Go to each page and find the movie release date
    baseURL = "https://en.wikipedia.org/wiki/"
    response = requests.get(baseURL+url)
    soup = BeautifulSoup(response.text)
    movies[movie]['release dates'] = release_dates(soup)
    movies[movie]['budget'] = budget(soup)
  
    time.sleep(2) # Don't spam wikipedia TOOOO much....


In [ ]:
#with open("movies2015.pkl", "wb") as f:
#    pickle.dump(movies, f)

with open("movies2016.pkl", "r") as f:
    movies = pickle.load(f)

In [ ]:
for movie, data in movies.iteritems():
    print movie
    article = data['url']
    dates = data['release dates']
    dates = [parser.parse(date.split("(")[1].split(")")[0]) for date in data['release dates']]
    end_date = max(dates)
    start_date = min(dates) - timedelta(weeks = 12)
    
    
    unreleased = max(dates) > datetime.now() # Movie isn't out yet... can't do analysis
    nodata = (min(dates) - timedelta(weeks = 12)) < parser.parse("20150801") # Movie released before wikiViews existed
    already_saved = movie+".csv" in os.listdir("WikiVisits/") # already saved movie data!
    if unreleased or nodata or already_saved:
        continue

    url = ("https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/"
          "all-access/all-agents/%s/daily/%s/%s") % (article, 
                                                     str(start_date).split(" ")[0].replace("-", ""), 
                                                     str(end_date).split(" ")[0].replace("-", ""))

    response = requests.get(url)
    
    try:
        items = response.json()['items']
        views = [[item['timestamp'], item['views']] for item in items]

        with open("WikiVisits/"+movie+".csv", "w") as f:
            writer = csv.writer(f)
            writer.writerow(["timestamp", "views"])
            writer.writerows(views)
        print "Saved " + movie
    except:
        print movie + " failed"

    time.sleep(random.randint(2,5))

In [209]:
dfs = []
year = 2015
for page in range(1,8):
    df = pd.read_html("http://www.boxofficemojo.com/yearly/chart/?page=%d&view=releasedate&view2=domestic&yr=%s&p=.htm"%(page,year))
    dfs.append(df[3].iloc[5:, 1:8])
    time.sleep(2)
    

In [219]:
df  = pd.concat(dfs, ignore_index=True) 
df.columns = ["title", "studio", "gross", "total theaters", "opening gross", "opening theaters", "release"]

def add_year(s):
    try:
        return s + "/%s"%str(year)[2:]
    except:
        return None
    
df.release = df.release.apply(add_year)
    
df.to_csv("finnancial%d.csv"%year, encoding='utf-8')

In [406]:
df = pd.concat([pd.read_csv("finnancial2015.csv"), pd.read_csv("finnancial2016.csv")], ignore_index=True)
df = df.iloc[:, 1:]
#finance = finance.set_index("title")
#finance.info()
#movie_list = os.listdir("WikiVisits/")
#df = finance.loc[[title.split(".")[0] for title in movie_list]].iloc[:, 1:].reset_index()
#df.head()
#df.info()

def strip_dollar_sign(s):
    try:
        return int(s.replace("$", "").replace(",",""))
    except (AttributeError, ValueError):
        return None
    
def to_datetime(t):
    try:
        return datetime.strptime(t, "%m/%d/%y")
    except:
        return None


df['opening gross'] = df['opening gross'].apply(strip_dollar_sign)
df['gross'] = df.gross.apply(strip_dollar_sign)
df.release = df.release.apply(to_datetime)
df['opening theaters'] = pd.to_numeric(df['opening theaters'], errors="coerce")
df['total theaters'] = pd.to_numeric(df['total theaters'], errors="coerce")

#df.sort_values("gross", ascending=False).head(10)

"""
wikivol = []
for movie in df.title[1:]:
    wikiviews = pd.read_csv("WikiVisits/%s"%(movie+".csv"))
    wikiviews.timestamp = wikiviews.timestamp.apply(lambda d: datetime.strptime(str(d/100), "%Y%m%d"))
    vol = wikiviews[wikiviews.timestamp > (wikiviews.timestamp.iloc[-1] - timedelta(weeks=2))].views.sum()
    wikivol.append(vol)
    
df.loc[1:, "wiki views"] = wikivol
"""
df.head()

,title,studio,gross,total theaters,opening gross,opening theaters,release
0,Star Wars: The Force Awakens,BV,935832841,4134,247966675,4134,2015-12-18
1,Jurassic World,Uni.,652270625,4291,208806270,4274,2015-06-12
2,Avengers: Age of Ultron,BV,459005868,4276,191271109,4276,2015-05-01
3,Inside Out,BV,356461711,4158,90440272,3946,2015-06-19
4,Furious 7,Uni.,353007020,4022,147187040,4004,2015-04-03


In [407]:
wikiviews = {}

In [ ]:
from collections import defaultdict
import json

movies = pickle.load(open("movies.pkl"))

for movie_title, series in df.set_index('title').loc[[title for title in df.title if title in movies]].iterrows():
    article_name = movies[movie_title]['url']
    
    if movie_title not in wikiviews:
        release_date = series.release.date()
        temp_dict = defaultdict(dict)
        for date in [release_date, release_date - timedelta(weeks=4)]:
            date = str(date.year)+"%02d" % date.month
            response = requests.get('http://stats.grok.se/json/en/%s/%s'%(date, urllib.quote(article_name)))
            if response.ok:
                response = response.json()
                temp_dict['daily_views'].update(response['daily_views'])
            else:
                print "Warning! Failed:", movie_title, date
                continue
                
        temp_dict['title'] = response['title']
        wikiviews[movie_title] = temp_dict

        with open("log.txt", "a") as log:
            json.dump(response, log)
        
        print movie_title
        
        time.sleep(1)



Spectre
Pitch Perfect 2
The Revenant
Ant-Man
Hotel Transylvania 2
Fifty Shades of Grey
The SpongeBob Movie: Sponge Out of Water
Straight Outta Compton
San Andreas
Mad Max: Fury Road
Daddy's Home
The Divergent Series: Insurgent
The Peanuts Movie
The Good Dinosaur
Spy
Trainwreck
Creed
Tomorrowland
Get Hard
Sisters
Alvin and the Chipmunks: The Road Chip
Maze Runner: The Scorch Trials
Ted 2
Goosebumps
Pixels
The Intern
Bridge of Spies
Paul Blart: Mall Cop 2
The Big Short
War Room
Magic Mike XXL
The Visit
The Wedding Ringer
Black Mass
Vacation
The Perfect Guy
Joy
Fantastic Four
The Hateful Eight
Southpaw
Jupiter Ascending
Sicario
Spotlight
McFarland, USA
The Night Before
Krampus
The Age of Adaline
Brooklyn
The Longest Ride
The Boy Next Door
Pan
Hot Pursuit
Woman in Gold
The Second Best Exotic Marigold Hotel
Entourage
Paper Towns
Chappie
Crimson Peak
A Walk in the Woods
Sinister 2
The Last Witch Hunter
No Escape
Ricki and the Flash
Run All Night
Love the Coopers
The Lazarus Effect
Ex Machina

In [ ]:
wikiviews

In [338]:
pickle.dump(wikiviews, open("wikiviews.pkl", "w"))

In [393]:
#df = df.set_index("title")
def to_datetime(t):
    try:
        return datetime.strptime(t, '%Y-%m-%d')
    except ValueError:
        return pd.NaT
    
df['wiki views'] = np.NaN
for movie in wikiviews.keys():
    views = pd.DataFrame(wikiviews[movie]['daily_views'].items(), columns=['time', 'views'])
    
    if len(views) == 0:
        print "Warning: %s has no data" % movie
        continue
        
    views.time = views.time.apply(to_datetime)
    
    views.sort_values("time")
    df.loc[movie, 'wiki views'] = views[views.time > (views.time.iloc[-1] - timedelta(weeks=4))].views.sum()


In [398]:
df.to_csv("masterDF.csv")

In [400]:
wikiviews["The Second Mother"]

defaultdict(dict,
            {'daily_views': {u'2015-07-01': 0,
              u'2015-07-02': 4,
              u'2015-07-03': 1,
              u'2015-07-04': 0,
              u'2015-07-05': 1,
              u'2015-07-06': 1,
              u'2015-07-07': 1,
              u'2015-07-08': 0,
              u'2015-07-09': 1,
              u'2015-07-10': 1,
              u'2015-07-11': 0,
              u'2015-07-12': 0,
              u'2015-07-13': 4,
              u'2015-07-14': 0,
              u'2015-07-15': 1,
              u'2015-07-16': 4,
              u'2015-07-17': 0,
              u'2015-07-18': 1,
              u'2015-07-19': 0,
              u'2015-07-20': 3,
              u'2015-07-21': 1,
              u'2015-07-22': 1,
              u'2015-07-23': 4,
              u'2015-07-24': 0,
              u'2015-07-25': 0,
              u'2015-07-26': 0,
              u'2015-07-27': 9,
              u'2015-07-28': 5,
              u'2015-07-29': 1,
              u'2015-07-30': 1,
       

In [401]:
wikiviews

{'10 Cloverfield Lane': defaultdict(dict,
             {'daily_views': {}, 'title': u'10 Cloverfield Lane'}),
 '13 Hours: The Secret Soldiers of Benghazi': defaultdict(dict,
             {'daily_views': {u'2015-12-01': 1717,
               u'2015-12-02': 1449,
               u'2015-12-03': 1402,
               u'2015-12-04': 1513,
               u'2015-12-05': 1139,
               u'2015-12-06': 1183,
               u'2015-12-07': 1519,
               u'2015-12-08': 1638,
               u'2015-12-09': 1735,
               u'2015-12-10': 2011,
               u'2015-12-11': 1806,
               u'2015-12-12': 1372,
               u'2015-12-13': 1428,
               u'2015-12-14': 2288,
               u'2015-12-15': 2365,
               u'2015-12-16': 4982,
               u'2015-12-17': 3040,
               u'2015-12-18': 4546,
               u'2015-12-19': 2522,
               u'2015-12-20': 3938,
               u'2015-12-21': 5549,
               u'2015-12-22': 5128,
               u'20